# Imports

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import datetime

# Load data

In [4]:

X = pickle.load(open("X_binary.pickle", "rb"))
y = pickle.load(open("y_binary.pickle", "rb"))

X = X/255.0

# Training & Optimization

In [23]:


dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            MODEL_NAME = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense@{datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}"
            logdir = f"logs/{MODEL_NAME}"
            tensorboard = TensorBoard(logdir, histogram_freq=1)

            print(f"Now training: {MODEL_NAME}")

            # Train the model with the specified hyperparameters
            model = Sequential()

            model.add(Conv2D(64, (3, 3), input_shape = X.shape[1:])) # 200 x 200 x 3
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for count in range(conv_layer - 1):
                model.add(Conv2D(64, (3, 3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for count in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

            model.fit(X, y, batch_size=32, validation_split=0.1, epochs=10, callbacks=[tensorboard])
            model.save(f'{MODEL_NAME}.model')

Now training: {conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense@{datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}
Epoch 1/10
 8/76 [==>...........................] - ETA: 37s - loss: 8.5456 - accuracy: 0.4136

KeyboardInterrupt: 

# Observing the training process with TensorBoard
To get TensorBoard running, run the following command __in your terminal__ at the same directory that houses the notebook script:
```bash
tensorboard --logdir=logs/
```
The above command should spin up a TensorBoard client accessible via `localhost:6006`